# 信用卡欺诈检测

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
%matplotlib inline

In [ ]:
  print(skl.__version__)

In [ ]:
data = pd.read_csv("creditcard.csv")
data.head()
# Time与分析过程没什么关系，V1-V28，是被银行处理过的数据，Amount是消费金额，Class=0是正样本，Class=1是负样本

In [ ]:
# 用value_counts，计算一下，正负样本的数量
count_classes = pd.value_counts(data['Class'], sort = True).sort_index()
count_classes.plot(kind = 'bar')
plt.style.use(['ggplot','bmh'])
plt.title("Fraud class histogram")
plt.xlabel("Class")
plt.ylabel("Frequency")

过采样，下采样

sklearn，机器训练和预处理

preprocessing，预处理模块。StandardScaler，标准化函数

In [ ]:
from sklearn.preprocessing import StandardScaler
# 生成新的列，normAmount
# reshape:[3,2],reshape(-1,3),[2,3]
data['normAmount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))
# 清除Time列和Amount列
data = data.drop(['Time','Amount'],axis=1)
data.head()

开始下采样，让0和1一样少

In [ ]:
X = data.loc[:, data.columns != 'Class']
y = data.loc[:, data.columns == 'Class']

# 取出Class等于1的样本个数
number_records_fraud = len(data[data.Class == 1])
#print(number_records_fraud)
# 取出Class等于1的样本的index值
fraud_indices = np.array(data[data.Class == 1].index)

# # 取出Class等于0的样本的index值
normal_indices = data[data.Class == 0].index

# 在normal_indices的里面，进行一个随机选择（np.random.choic），选number_records_fraud个
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
# 取出的数据，转换成np.array结构
random_normal_indices = np.array(random_normal_indices)

# 0和1的数据，进行合并np.concatenate
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# 最后得到一个下采样数据under_sample_data
under_sample_data = data.iloc[under_sample_indices,:]
# 把under_sample_data，分成两部分，
X_undersample = under_sample_data.loc[:, under_sample_data.columns != 'Class']
y_undersample = under_sample_data.loc[:, under_sample_data.columns == 'Class']

# 显示正负样本的比例，显示新样本的整体数量
print("Percentage of normal transactions: ", len(under_sample_data[under_sample_data.Class == 0])/len(under_sample_data))
print("Percentage of fraud transactions: ", len(under_sample_data[under_sample_data.Class == 1])/len(under_sample_data))
print("Total number of transactions in resampled data: ", len(under_sample_data))

交叉验证

举例：

1. 用80%数据做训练，用20%数据做验证，训练集和测试集
2. 把训练集做一个平均的切分，切分成1，2，3号数据集
    1. 用1+2训练，把3作为验证集
    2. 用1+3训练，把2作为验证集
    3. 用2+3训练，把1作为验证集
3. 让模型的评估效果是可信的，求稳！
4. 建立模型时候，最难的就是调 **参数**

In [ ]:
# 引入交叉验证函数train_test_split
from sklearn.model_selection import train_test_split
# test_size，设置测试集的比例。 random_state，指定随机状态。
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 0)

print("Number transactions train dataset: ", len(X_train))
print("Number transactions test dataset: ", len(X_test))
print("Total number of transactions: ", len(X_train)+len(X_test))

# 对下采样数据集，进行同样的操作
X_train_undersample, X_test_undersample, y_train_undersample, y_test_undersample = train_test_split(X_undersample
                                                                                                   ,y_undersample
                                                                                                   ,test_size = 0.3
                                                                                                   ,random_state = 0)
print("")
print("Number transactions train dataset: ", len(X_train_undersample))
print("Number transactions test dataset: ", len(X_test_undersample))
print("Total number of transactions: ", len(X_train_undersample)+len(X_test_undersample))

# Recall

![Recall](Recall.png)

In [ ]:
# 使用Recall = TP/(TP+FN)，来评估模型的准确性，而不使用精度。
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix,recall_score,classification_report 

In [ ]:
def printing_Kfold_scores(x_train_data,y_train_data):
    fold = KFold(5,shuffle=False) 

    # 正则化惩罚项，通过系数，进行交叉验证，
    c_param_range = [0.01,0.1,1,10,100]

    results_table = pd.DataFrame(index = range(len(c_param_range),2), columns = ['C_parameter','Mean recall score'])
    results_table['C_parameter'] = c_param_range

    # the k-fold will give 2 lists: train_indices = indices[0], test_indices = indices[1]
    j = 0
    # 第一个循环，找到最佳的惩罚系数
    for c_param in c_param_range:
        print('-------------------------------------------')
        print('C parameter: ', c_param)
        print('-------------------------------------------')
        print('')

        recall_accs = []
        # 第二个循环，交叉验证
        for iteration, indices in enumerate(fold.split(x_train_data),start=1):

            # Call the logistic regression model with a certain C parameter
            # 直接调用逻辑回归LogisticRegression
            lr = LogisticRegression(C = c_param, penalty = 'l1', solver='liblinear')

            # Use the training data to fit the model. In this case, we use the portion of the fold to train the model
            # with indices[0]. We then predict on the portion assigned as the 'test cross validation' with indices[1]
            # 把训练数据传入lr
            lr.fit(x_train_data.iloc[indices[0],:],y_train_data.iloc[indices[0],:].values.ravel())

            # Predict values using the test indices in the training data
            # 进行预测验证
            y_pred_undersample = lr.predict(x_train_data.iloc[indices[1],:].values)

            # Calculate the recall score and append it to a list for recall scores representing the current c_parameter
            # 计算召回率
            recall_acc = recall_score(y_train_data.iloc[indices[1],:].values,y_pred_undersample)
            # 把召回率，放入到集合中
            recall_accs.append(recall_acc)
            print('Iteration ', iteration,': recall score = ', recall_acc)

        # The mean value of those recall scores is the metric we want to save and get hold of.
        results_table.loc[j,'Mean recall score'] = np.mean(recall_accs)
        j += 1
        print('')
        print('Mean recall score ', np.mean(recall_accs))
        print('')

    best_c = results_table.loc[results_table['Mean recall score'].astype('float64').idxmax()]['C_parameter']
    
    # Finally, we can check which C parameter is the best amongst the chosen.
    print('*********************************************************************************')
    print('Best model to choose from cross validation is with C parameter = ', best_c)
    print('*********************************************************************************')
    
    return best_c

In [ ]:
best_c = printing_Kfold_scores(X_train_undersample,y_train_undersample)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
import itertools
lr = LogisticRegression(C = best_c, penalty = 'l1', solver='liblinear')
lr.fit(X_train_undersample,y_train_undersample.values.ravel())
y_pred_undersample = lr.predict(X_test_undersample.values)

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test_undersample,y_pred_undersample)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

In [ ]:
lr = LogisticRegression(C = best_c, penalty = 'l1', solver='liblinear')
lr.fit(X_train_undersample,y_train_undersample.values.ravel())
# 直接预测
y_pred = lr.predict(X_test.values)

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test,y_pred)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

下采样，带来的“误杀”数量，有点儿多。

In [ ]:
# 原始数据集，直接交叉验证
best_c = printing_Kfold_scores(X_train,y_train)

In [ ]:
lr = LogisticRegression(C = best_c, penalty = 'l1', solver='liblinear')
lr.fit(X_train,y_train.values.ravel())
# 直接预测
y_pred_undersample = lr.predict(X_test.values)

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test,y_pred_undersample)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

用原始数据的混淆矩阵，recall值，比较低。

在不同的sigmoid中，设置不同的阈值，查看

In [ ]:
lr = LogisticRegression(C = 0.01, penalty = 'l1', solver='liblinear')
lr.fit(X_train_undersample,y_train_undersample.values.ravel())
# predict_proba，预测概率值
y_pred_undersample_proba = lr.predict_proba(X_test_undersample.values)
# 希望预测的阈值
thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

plt.figure(figsize=(10,10))

j = 1
for i in thresholds:
    y_test_predictions_high_recall = y_pred_undersample_proba[:,1] > i
    
    plt.subplot(3,3,j)
    j += 1
    
    # Compute confusion matrix
    cnf_matrix = confusion_matrix(y_test_undersample,y_test_predictions_high_recall)
    np.set_printoptions(precision=2)

    print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

    # Plot non-normalized confusion matrix
    class_names = [0,1]
    plot_confusion_matrix(cnf_matrix
                          , classes=class_names
                          , title='Threshold >= %s'%i) 

使用逻辑回归建模时，需要考虑实际情况，来设定阈值，同时保证recall值和精度，都在可接受的范围内。

smote算法

![smote](smote.png)

In [ ]:
import pandas as pd
# pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
credit_cards=pd.read_csv('creditcard.csv')

columns=credit_cards.columns
# The labels are in the last column ('Class'). Simply remove it to obtain features columns
features_columns=columns.delete(len(columns)-1)

features=credit_cards[features_columns]
labels=credit_cards['Class']

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, 
                                                                            labels, 
                                                                            test_size=0.2, 
                                                                            random_state=0)

In [ ]:
# 使用smote算法，每次生成数据都保持一致random_state=0
oversampler=SMOTE(random_state=0)
# 只针对训练集，进行生成，不动测试集
os_features,os_labels=oversampler.fit_sample(features_train,labels_train)

In [ ]:
len(os_labels[os_labels==1])

In [ ]:
os_features = pd.DataFrame(os_features)
os_labels = pd.DataFrame(os_labels)
best_c = printing_Kfold_scores(os_features,os_labels)

In [ ]:
lr = LogisticRegression(C = best_c, penalty = 'l1', solver='liblinear')
lr.fit(os_features,os_labels.values.ravel())
y_pred = lr.predict(features_test.values)

# Compute confusion matrix
cnf_matrix = confusion_matrix(labels_test,y_pred)
np.set_printoptions(precision=2)

print("Recall metric in the testing dataset: ", cnf_matrix[1,1]/(cnf_matrix[1,0]+cnf_matrix[1,1]))

# Plot non-normalized confusion matrix
class_names = [0,1]
plt.figure()
plot_confusion_matrix(cnf_matrix
                      , classes=class_names
                      , title='Confusion matrix')
plt.show()

样本不均衡的时候，首选过采样，毕竟对于预测，数据越多越有利。